In [5]:
import pandas as pd
import datetime

In [6]:
data_path = "/Users/jurajkapasny/Data/energy_hack/"

### Load data

In [3]:
om_info = pd.read_excel(data_path + "Energyhack2_data_update.xlsx", sheet_name="1. Odberné miesta")
om_spotreba = pd.read_excel(data_path + "Energyhack2_data_update.xlsx", sheet_name="2. Spotreba 150 OM")
vyroba = pd.read_excel(data_path + "Energyhack2_data_update.xlsx", sheet_name="3. Výroba a počet FVE")

### clean and transform spotreba

#### spotreba

In [4]:
om_spotreba.drop("Veličina", axis = 1, inplace = True)

In [5]:
om_spotreba = om_spotreba.set_index("Číslo OM")
om_spotreba["Dátum a čas"] = om_spotreba["Dátum a čas"].astype(str)

In [6]:
om_spotreba.rename(columns={datetime.datetime(1900, 1, 1, 0, 0):'00:00:00'}, inplace=True)

In [7]:
transformed_data = []
for i in range(1,151):
    temp = pd.DataFrame(om_spotreba[om_spotreba.index == i].set_index("Dátum a čas").stack(), columns=["spotreba"])
    temp["om"] = i
    transformed_data.append(temp)
om_spotreba_tr = pd.concat(transformed_data)

In [8]:
om_spotreba_tr.to_csv(data_path + "spotreba_prepared.csv", sep = ";")

#### vyroba

In [117]:
vyroba.head()

,Inštalovaný výkon FVE (kWp),2,2.5,3.15,4,5,7.2,9.2,Unnamed: 8,Unnamed: 9,...,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42
0,Dátum a čas,Výroba FVE (kWh),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-01 00:00:00,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,vyroba,3000.0,NaN,NaN,spotreba bez fve,3000.0,360.0,NaN,spotreba bez fve,2000.0
2,2016-01-01 00:15:00,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,Spotreba,1000.0,NaN,NaN,Spotreba z FVE,1000.0,120.0,NaN,Spotreba z FVE,1000.0
3,2016-01-01 00:30:00,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,Usetrene,120.0,NaN,NaN,Usetrene,10.0,50.0,NaN,Usetrene,NaN
4,2016-01-01 00:45:00,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,zarobene,90.0,NaN,NaN,NaN,NaN,70.0,NaN,NaN,NaN


In [118]:
to_drop = []
for cl in vyroba.columns:
    if str(cl).find("Unnamed") != -1:
        to_drop.append(cl)
vyroba.drop(to_drop, axis=1, inplace=True)

In [119]:
vyroba.drop(0, axis = 0, inplace=True)

In [120]:
vyroba.rename(columns={"Inštalovaný výkon FVE (kWp)":'time'}, inplace=True)

In [121]:
vyroba = vyroba.set_index("time")

In [122]:
vyroba.head()

,2,2.5,3.15,4,5,7.2,9.2
time,,,,,,,
2016-01-01 00:00:00,0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 00:15:00,0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 00:30:00,0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 00:45:00,0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-01 01:00:00,0,0.0,0.0,0.0,0.0,0.0,0.0


In [123]:
vyroba.to_csv(data_path + "vyroba_prepared.csv", sep = ";")

#### om info

In [125]:
om_info.drop(0, axis=0,inplace=True)

In [126]:
om_info.head()

,Číslo OM,Zapojenie,Druh tarify,Inštalovaný výkon FVE (kWp)
1,1,A,D2,2
2,2,A,D2,7.2
3,3,A,D2,2
4,4,A,D1,0
5,5,C,D4,9.2


In [127]:
om_info.to_csv(data_path + "om_info_prepared.csv", sep = ";")

#### Cena nakup zo siete

In [7]:
druh_tarify = ["D1","D2","D3","D4","D5"]
hodiny = [i for i in range(24)]

In [8]:
index = pd.MultiIndex.from_product([druh_tarify, hodiny], names = ["druh_tarify", "hodiny"])
price_nakup_zo_siete = pd.DataFrame(index = index).reset_index()


In [11]:
price_nakup_zo_siete["price"] = [0.119989,0.119989,0.119989,0.119989,
                                0.119989,0.119989,0.119989,0.119989,
                                0.119989,0.119989,0.119989,0.119989,
                                0.119989,0.119989,0.119989,0.119989,
                                0.119989,0.119989,0.119989,0.119989,
                                0.119989,0.119989,0.119989,0.119989,
                                0.100094,0.100094,0.100094,0.100094,
                                0.100094,0.100094,0.100094,0.100094,
                                0.100094,0.100094,0.100094,0.100094,
                                0.100094,0.100094,0.100094,0.100094,
                                0.100094,0.100094,0.100094,0.100094,
                                0.100094,0.100094,0.100094,0.100094,
                                0.083694,0.083694,0.083694,0.083694,
                                0.103394,0.103394,0.103394,0.083694,
                                0.083694,0.103394,0.103394,0.103394,
                                0.103394,0.103394,0.083694,0.083694,
                                0.083694,0.103394,0.103394,0.103394,
                                0.103394,0.103394,0.103394,0.083694,
                                0.075168,0.075168,0.075168,0.075168,
                                0.075168,0.075168,0.098168,0.098168,
                                0.098168,0.098168,0.098168,0.098168,
                                0.098168,0.098168,0.098168,0.098168,
                                0.098168,0.098168,0.098168,0.098168,
                                0.098168,0.098168,0.075168,0.075168,
                                0.099668,0.077868,0.077868,0.077868,
                                0.077868,0.077868,0.077868,0.077868,
                                0.077868,0.099668,0.077868,0.099668,
                                0.077868,0.077868,0.077868,0.077868,
                                0.077868,0.077868,0.077868,0.077868,
                                0.077868,0.077868,0.099668,0.077868
                                ]

In [13]:
price_nakup_zo_siete.to_csv(data_path + "price_zo_siete.csv", sep = ";", index = False)

#### Cena nakup z komunity

In [14]:
price_nakup_z_komunity = pd.read_excel(data_path + "Energyhack2_data_update.xlsx", sheet_name="B. Cena nákup komunita")

In [22]:
price_nakup_z_komunity_cl = price_nakup_z_komunity.iloc[1:,0:2].reset_index(drop=True)

In [34]:
add_on = price_nakup_z_komunity.iloc[1:7,[3,5]]
add_on.columns = ["tarifa","cena"]
add_on.drop(1,axis = 0, inplace=True)

In [39]:
price_nakup_z_komunity_cl.columns = ["timestamp","main_price"]
price_nakup_z_komunity_cl.timestamp = pd.to_datetime(price_nakup_z_komunity_cl.timestamp)
price_nakup_z_komunity_cl.main_price = price_nakup_z_komunity_cl.main_price / 1000

In [56]:
price_nakup_z_komunity_cl.to_csv(data_path + "price_z_komunity.csv", sep = ";", index = False)
add_on.to_csv(data_path + "add_on_price_z_komunity.csv", sep = ";", index = False)

#### Cena predaj do komunity

In [48]:
price_predaj_do_komunity = pd.read_excel(data_path + "Energyhack2_data_update.xlsx", sheet_name="C. Cena predaj komunita")

In [49]:
price_predaj_do_komunity = price_predaj_do_komunity.iloc[1:,0:2].reset_index(drop=True)

In [50]:
price_predaj_do_komunity.columns = ["timestamp","main_price"]
price_predaj_do_komunity.timestamp = pd.to_datetime(price_predaj_do_komunity.timestamp)
price_predaj_do_komunity.main_price = (price_predaj_do_komunity.main_price / 1000) - 0.008

In [51]:
price_predaj_do_komunity.head()

,timestamp,main_price
0,2016-01-01 00:00:00,0.0249
1,2016-01-01 00:15:00,0.02501
2,2016-01-01 00:30:00,0.02512
3,2016-01-01 00:45:00,0.02523
4,2016-01-01 01:00:00,0.02534


In [52]:
price_predaj_do_komunity.to_csv(data_path + "price_do_komunity.csv", sep = ";", index = False)

#### Cena predaj do siete

In [43]:
price_predaj_do_siete = pd.read_excel(data_path + "Energyhack2_data_update.xlsx", sheet_name="D. Cena predaj sieť")

In [53]:
price_predaj_do_siete = price_predaj_do_siete.iloc[1:,0:2].reset_index(drop=True)

price_predaj_do_siete.columns = ["timestamp","main_price"]
price_predaj_do_siete.timestamp = pd.to_datetime(price_predaj_do_siete.timestamp)
price_predaj_do_siete.main_price = (price_predaj_do_siete.main_price / 1000) - 0.012

In [54]:
price_predaj_do_siete.head()

,timestamp,main_price
0,2016-01-01 00:00:00,0.0209
1,2016-01-01 00:15:00,0.02101
2,2016-01-01 00:30:00,0.02112
3,2016-01-01 00:45:00,0.02123
4,2016-01-01 01:00:00,0.02134


In [55]:
price_predaj_do_siete.to_csv(data_path + "price_do_siete.csv", sep = ";", index = False)